In [ ]:
# %load conv_layers.py
import numpy as np
from nndl.layers import *
import pdb


def conv_forward_naive(x, w, b, conv_param):
  """
  A naive implementation of the forward pass for a convolutional layer.

  The input consists of N data points, each with C channels, height H and width
  W. We convolve each input with F different filters, where each filter spans
  all C channels and has height HH and width HH.

  Input:
  - x: Input data of shape (N, C, H, W)
  - w: Filter weights of shape (F, C, HH, WW)
  - b: Biases, of shape (F,)
  - conv_param: A dictionary with the following keys:
    - 'stride': The number of pixels between adjacent receptive fields in the
      horizontal and vertical directions.
    - 'pad': The number of pixels that will be used to zero-pad the input.

  Returns a tuple of:
  - out: Output data, of shape (N, F, H', W') where H' and W' are given by
    H' = 1 + (H + 2 * pad - HH) / stride
    W' = 1 + (W + 2 * pad - WW) / stride
  - cache: (x, w, b, conv_param)
  """
  out = None
  pad = conv_param['pad']
  stride = conv_param['stride']

  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the forward pass of a convolutional neural network.
  #   Store the output as 'out'.
  #   Hint: to pad the array, you can use the function np.pad.
  # ================================================================ #
  pad_size = ((0,0), (0,0), (pad, pad), (pad, pad))
  pad_x = np.pad(x, pad_size, mode = 'constant', constant_values=0)
  N, C, H, W = x.shape
  F, C, HH, WW = w.shape
  output_H = int(1 + (H + 2*pad - HH) / stride)
  output_W = int(1 + (W + 2*pad - WW) / stride)
  
  out = np.zeros((N, F, output_H, output_W))
  for n in range(N):
    for h in range(output_H):
      for ww in range(output_W):
        # get the (k, 1, h*stride : h*stride+HH, w*stride : w*stride+WW) part
        x__conv_part = pad_x[n, :, h*stride:h*stride+HH, ww*stride:ww*stride+WW]
        x_after_conv_part = x__conv_part * w
        out[n, :, h, ww] = np.sum(x_after_conv_part, axis = (1,2,3))
  out = out + b[None, :, None, None]
  pass

  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ #
    
  cache = (x, w, b, conv_param)
  return out, cache


def conv_backward_naive(dout, cache):
  """
  A naive implementation of the backward pass for a convolutional layer.

  Inputs:
  - dout: Upstream derivatives.
  - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

  Returns a tuple of:
  - dx: Gradient with respect to x
  - dw: Gradient with respect to w
  - db: Gradient with respect to b
  """
  dx, dw, db = None, None, None

  N, F, out_height, out_width = dout.shape
  x, w, b, conv_param = cache
  
  stride, pad = [conv_param['stride'], conv_param['pad']]
  xpad = np.pad(x, ((0,0), (0,0), (pad,pad), (pad,pad)), mode='constant')
  num_filts, _, f_height, f_width = w.shape

  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the backward pass of a convolutional neural network.
  #   Calculate the gradients: dx, dw, and db.
  # ================================================================ #
  db = np.sum(dout, axis = (0,2,3))
  dw = np.zeros(w.shape)
  dx_pad = np.zeros(xpad.shape)
  for n in range(N):
    for hh in range(out_height):
      for ww in range(out_width):
        dx_pad[n, :, hh*stride:hh*stride + f_height, ww*stride:ww*stride + f_width] += np.sum((w*(dout[n, :, hh, ww])[:,None ,None, None]), axis=0)
        dw += xpad[n,:,hh*stride:hh*stride+f_height,ww*stride:ww*stride+f_width]*(dout[n,:,hh,ww])[:,None,None,None]
  dx = dx_pad[:,:,pad:-pad,pad:-pad]
  pass

  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ #

  return dx, dw, db


def max_pool_forward_naive(x, pool_param):
  """
  A naive implementation of the forward pass for a max pooling layer.

  Inputs:
  - x: Input data, of shape (N, C, H, W)
  - pool_param: dictionary with the following keys:
    - 'pool_height': The height of each pooling region
    - 'pool_width': The width of each pooling region
    - 'stride': The distance between adjacent pooling regions

  Returns a tuple of:
  - out: Output data
  - cache: (x, pool_param)
  """
  out = None
  
  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the max pooling forward pass.
  # ================================================================ #
  N,C,H,W = x.shape
  HH, WW, S = pool_param['pool_height'], pool_param['pool_width'], pool_param['stride']
  out_H = int(1 + (H - HH) / S)
  out_W = int(1 + (W - WW) / S)
  out = np.zeros((N,C,out_H,out_W))

  for i in range(out_H):
    for j in range(out_W):
      x_pooling_part = x[:,:,i*S : i*S+HH, j*S : j*S+WW]
      out[:, :, i, j] = np.max(x_pooling_part, axis = (2,3))
  pass
  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ # 
  cache = (x, pool_param)
  return out, cache

def max_pool_backward_naive(dout, cache):
  """
  A naive implementation of the backward pass for a max pooling layer.

  Inputs:
  - dout: Upstream derivatives
  - cache: A tuple of (x, pool_param) as in the forward pass.

  Returns:
  - dx: Gradient with respect to x
  """
  dx = None
  x, pool_param = cache
  pool_height, pool_width, stride = pool_param['pool_height'], pool_param['pool_width'], pool_param['stride']

  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the max pooling backward pass.
  # ================================================================ #
  H,W = x.shape[2],x.shape[3]
  HH,WW,S = pool_height, pool_width, stride
  out_H = int(1 + (H - HH) / S)
  out_W = int(1 + (W - WW) / S)
  dx = np.zeros_like(x)
  for i in range(out_H):
    for j in range(out_W):
      x_pooling_part = x[:,:,i*stride : i*S+HH,j*S : j*S+WW]
      pooling_idx = np.max(x_pooling_part, axis = (2,3))
      pooling_mask = np.zeros_like(x_pooling_part)
      pooling_mask = pooling_idx[:,:,None,None]==x_pooling_part
      dx[:,:,i*S : i*S+HH,j*S : j*S+WW] += pooling_mask*(dout[:,:,i,j])[:,:,None,None]
  pass

  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ # 

  return dx

def spatial_batchnorm_forward(x, gamma, beta, bn_param):
  """
  Computes the forward pass for spatial batch normalization.
  
  Inputs:
  - x: Input data of shape (N, C, H, W)
  - gamma: Scale parameter, of shape (C,)
  - beta: Shift parameter, of shape (C,)
  - bn_param: Dictionary with the following keys:
    - mode: 'train' or 'test'; required
    - eps: Constant for numeric stability
    - momentum: Constant for running mean / variance. momentum=0 means that
      old information is discarded completely at every time step, while
      momentum=1 means that new information is never incorporated. The
      default of momentum=0.9 should work well in most situations.
    - running_mean: Array of shape (D,) giving running mean of features
    - running_var Array of shape (D,) giving running variance of features
    
  Returns a tuple of:
  - out: Output data, of shape (N, C, H, W)
  - cache: Values needed for the backward pass
  """
  out, cache = None, None

  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the spatial batchnorm forward pass.
  #
  #   You may find it useful to use the batchnorm forward pass you 
  #   implemented in HW #4.
  # ================================================================ #
  N,C,H,W = x.shape
#   out = np.zeros((N,C,H,W))
  #reshape X into shape: (N*H*W, C) so that we can do normalization on layers
  x_reshape = x.transpose((0,2,3,1)).reshape(N*H*W, C)
  out, cache = batchnorm_forward(x_reshape, gamma, beta, bn_param)
  out = out.reshape(N, H, W, C).transpose((0, 3, 1, 2))
  pass

  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ # 

  return out, cache


def spatial_batchnorm_backward(dout, cache):
  """
  Computes the backward pass for spatial batch normalization.
  
  Inputs:
  - dout: Upstream derivatives, of shape (N, C, H, W)
  - cache: Values from the forward pass
  
  Returns a tuple of:
  - dx: Gradient with respect to inputs, of shape (N, C, H, W)
  - dgamma: Gradient with respect to scale parameter, of shape (C,)
  - dbeta: Gradient with respect to shift parameter, of shape (C,)
  """
  dx, dgamma, dbeta = None, None, None

  # ================================================================ #
  # YOUR CODE HERE:
  #   Implement the spatial batchnorm backward pass.
  #
  #   You may find it useful to use the batchnorm forward pass you 
  #   implemented in HW #4.
  # ================================================================ #
  N,C,H,W = dout.shape
  dout = dout.transpose((0, 2, 3, 1)).reshape(N * H * W, C)
  dx, dgamma, dbeta = batchnorm_backward(dout, cache)
  dx = dx.reshape(N, H, W, C).transpose((0, 3, 1, 2))
  # ================================================================ #
  # END YOUR CODE HERE
  # ================================================================ # 

  return dx, dgamma, dbeta